In [2]:
#GENERATING THE DATA for DEMOGRAPHICS
import time
import requests
import pandas as pd
from pandas import json_normalize
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta

#datas a serem requeridas:
start_date='2021-04' #YYYY-MM
end_date='2023-09'  #YYYY-MM

start = datetime.strptime(start_date, "%Y-%m")
end = datetime.strptime(end_date, "%Y-%m")

def process_dataframe(input_df, company, vertical, state,platform):
    # Create a new DataFrame with only the desired columns
    processed_df = input_df[['date', 'visits', 'bounce_rate', 'pages_per_visit',
                             'visit_duration', 'page_views', 'share', 'unique_visitors']].copy()
    
    # Add new columns with the provided inputs
    processed_df['company'] = company
    processed_df['vertical'] = vertical
    processed_df['state'] = state
    processed_df['platform'] = platform
    
    return processed_df

def transform_dataframe(original_df, company, vertical, state, platform):
    metric_name = original_df.columns[1]
    transformed_df = pd.DataFrame()
    transformed_df['date'] = original_df['date']
    transformed_df['metric_name'] = metric_name
    transformed_df['metric_value'] = original_df[metric_name]
    transformed_df['company'] = company
    transformed_df['vertical'] = vertical
    transformed_df['state'] = state
    transformed_df['platform'] = platform
    return transformed_df

def transform_dataframe_0(original_df, company, vertical, state, platform,metric_name):
    transformed_df = pd.DataFrame()
    transformed_df['date'] = original_df['date']
    transformed_df['metric_name'] = metric_name
    transformed_df['metric_value'] = original_df[original_df.columns[0]]
    transformed_df['company'] = company
    transformed_df['vertical'] = vertical
    transformed_df['state'] = state
    transformed_df['platform'] = platform
    return transformed_df

def add_data_to_dataframe(df, json_data,age,gender,company,vertical):
    # Extrai os valores do JSON
    start_date = json_data['meta']['request']['start_date']
    share = json_data['share']
    pages_per_visit = json_data['pages_per_visit']
    bounce_rate = json_data['bounce_rate']
    visit_duration = json_data['visit_duration']

    # Adiciona os valores ao DataFrame
    df = df.append({
        'date': start_date,
        'audience_share': share,
        'pages_per_visit': pages_per_visit,
        'bounce_rate': bounce_rate,
        'visit_duration': visit_duration,
        'age': age,
        'gender': gender,
        'company': company,
        'vertical': vertical
    }, ignore_index=True)
    return df

company_urls =[
'olx.com.br', 
'quintoandar.com.br',
'chavesnamao.com.br',
'icarros.com.br',
'imovelweb.com.br',
'kavak.com',
'loft.com.br',
'mobiauto.com.br',
'vivareal.com.br',
'webmotors.com.br',
'zapimoveis.com.br',
'casamineira.com.br', 
'meucarronovo.com.br',
'enjoei.com.br',
'elo7.com.br',
'mercadolivre.com.br',
'shopee.com.br'
]
company_names =[
    'olx',
'quinto_andar',
'chaves_na_mao',
'icarros',
'imovelweb',
'kavak',
'loft',
'mobiauto',
'vivareal',
'webmotors',
'zap',
'casa_mineira',
'meu_carro_novo',
'enjoei',
'elo7',
'mercado_livre',
'shopee'
]
vertical_names =[
'FULL',
'REAL_ESTATE',
'FULL',
'AUTOS',
'REAL_ESTATE',
'AUTOS',
'REAL_ESTATE',
'AUTOS',
'REAL_ESTATE',
'AUTOS',
'REAL_ESTATE',
'REAL_ESTATE',
'AUTOS',
'GOODS',
'GOODS',
'GOODS',
'GOODS'
]
company_map = dict(zip(company_urls, company_names)) #mapear uma lista na outra
vertical_map = dict(zip(company_urls, vertical_names)) #mapear uma lista na outra

# Replace with your actual API key
API_KEY = "c165c953a9fd44908addfcccec2c8caa"
# Headers with authorization
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

#dataframe das métricas
columns = ['date', 'audience_share', 'pages_per_visit', 'bounce_rate', 'visit_duration','age','gender','company']
metrics_df =pd.DataFrame(columns=columns)
# Colunas do DataFrame final
columns = ['date', 'metric_name', 'metric_value', 'company', 'vertical', 'age','gender', 'device']
df_demographics_analysis = pd.DataFrame(columns=columns)


ages = ['18', '25', '35', '45', '55', '65']
genders=['male','female']
granularity='monthly'
device='Total'
sleep_time=1.0 
#################################################################################################################
while start <= end:
    first_day = start.strftime("%Y-%m-01")
    last_day = (start + relativedelta(day=31)).strftime("%Y-%m-%d")
                                                    ####################
                                                    ###Loop de coleta###
    for company_url in company_urls:                ####################
        company=company_map.get(company_url)
        vertical=vertical_map.get(company_url)
        for age in ages:
            for gender in genders:
                if gender=='male':
                    male_data='true'
                    female_data='false'
                if gender=='female':
                    male_data='false'
                    female_data='true'
                url='https://api.similarweb.com/v4/website/'+company_url+'/demographics_v2/groups?api_key=c165c953a9fd44908addfcccec2c8caa&start_date='+str(first_day)+'&end_date='+str(last_day)+'&country=br&age_buckets='+age+'&female_data='+female_data+'&male_data='+male_data+'&format=json'
                # Make the API request
                response = requests.get(url, headers={'api-key':'c165c953a9fd44908addfcccec2c8caa'})
                # Check if the request was successful
                if response.status_code == 200:
                    json_data= response.json()
                    # Process the data as needed
                    print('Data for '+company+'/'+device+'/'+age+'/'+gender+'/'+str(start)+' unique visitors desktop processed successfully!')
                else:
                    print('API request failed for '+company+' :', response.status_code)
                metrics_df=add_data_to_dataframe(metrics_df, json_data,age,gender,company,vertical)
                time.sleep(sleep_time)
    start = start + relativedelta(months=1)
df=metrics_df
for col in df.columns:
    if df[col].dtype == 'object':
        df = df.astype({col:'string'})
for col in df.columns:
    if df[col].dtype == 'float64':
        df = df.astype({col:'float32'})
df.rename(columns={'date': 'dt'}, inplace=True)
df['dt'] = pd.to_datetime(df['dt'])
df['year'] = df['dt'].dt.year
df['month'] = df['dt'].dt.month
df['device'] = device
metrics_df=df
age_mapping = {
    '18': '18-24',
    '25': '25-34',
    '35': '35-44',
    '45': '45-54',
    '55': '55-64',
    '65': '65+'
}

# Realiza a substituição na coluna "age"
metrics_df['age'] = metrics_df['age'].replace(age_mapping)

# Exibe o DataFrame resultante
print(metrics_df)

Data for olx/Total/18/male/2021-04-01 00:00:00 unique visitors desktop processed successfully!


/tmp/ipykernel_349/3383876958.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({


Data for olx/Total/18/female/2021-04-01 00:00:00 unique visitors desktop processed successfully!


/tmp/ipykernel_349/3383876958.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({


Data for olx/Total/25/male/2021-04-01 00:00:00 unique visitors desktop processed successfully!


...

Data for shopee/Total/65/male/2023-09-01 00:00:00 unique visitors desktop processed successfully!


/tmp/ipykernel_349/3383876958.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({


Data for shopee/Total/65/female/2023-09-01 00:00:00 unique visitors desktop processed successfully!


/tmp/ipykernel_349/3383876958.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({


             dt  audience_share  pages_per_visit  bounce_rate  visit_duration  \
0    2021-04-01        0.113610         9.924847     0.210117      617.774902   
1    2021-04-01        0.051524         9.531736     0.227799      606.185608   
2    2021-04-01        0.201495         9.817860     0.222233      618.611145   
3    2021-04-01        0.111304         9.303407     0.240437      601.777588   
4    2021-04-01        0.117196        10.081762     0.225834      664.196960   
...         ...             ...              ...          ...             ...   
6115 2023-09-01        0.078935         2.665984     0.758109      148.656158   
6116 2023-09-01        0.071548         2.635875     0.749590      155.953720   
6117 2023-09-01        0.061554         2.675161     0.762040      152.944626   
6118 2023-09-01        0.032253         2.667924     0.748818      158.754333   
6119 2023-09-01        0.024415         2.855726     0.750764      169.373123   

        age  gender company

In [14]:
metrics_df.to_csv('similarweb_demographics-'+start_date+'-'+end_date+'.csv', index=False)

In [4]:
import io
import os

#import boto3
import s3fs

import pandas as pd
import pyarrow as pa

In [5]:
from dataeng_services.session import Session

session_provider = Session().client("session-provider")
aws_auth_dict = session_provider.get_temporary_credentials().json()

In [6]:
AWS_S3_FOLDER = 's3://olxbr-dl-cross-estrategia-e-planejamento/prod/cross_strategy/similarweb_demographics'
# csv_file_name = 'dataframe_s'

date_cols = ['dt']
int_cols = []
partition_col = ['dt']

In [7]:
#LENDO ARQUIVO ORIGINAL
df = metrics_df
# df = pd.read_csv(csv_file_name+'.csv')

df.head()

,dt,audience_share,pages_per_visit,bounce_rate,visit_duration,age,gender,company,vertical,year,month,device
0,2021-04-01,0.113610,9.924847,0.210117,617.774902,18-24,male,olx,FULL,2021,4,Total
1,2021-04-01,0.051524,9.531736,0.227799,606.185608,18-24,female,olx,FULL,2021,4,Total
2,2021-04-01,0.201495,9.817860,0.222233,618.611145,25-34,male,olx,FULL,2021,4,Total
3,2021-04-01,0.111304,9.303407,0.240437,601.777588,25-34,female,olx,FULL,2021,4,Total
4,2021-04-01,0.117196,10.081762,0.225834,664.196960,35-44,male,olx,FULL,2021,4,Total


In [8]:
# Coverte as colunas de data
for col in date_cols:
    df[ col ] = pd.to_datetime(df[ col ], format='%Y-%m-%d') ## Date   

# Coverte as colunas int
df[ int_cols ] = df[ int_cols ].round()

# Confirma os tipos corretos das Var na AWS
pa_schema = pa.Schema.from_pandas(df)
pa_schema = pa.schema([
pa.field(field.name, pa.date32()) if field.name in date_cols else field
for field in pa_schema
])

In [9]:
#Convertendo e salvando na aws
# save converted files localy
if len( partition_col ) > 0 :
    df.to_parquet(
            AWS_S3_FOLDER,
            schema=pa_schema,
            partition_cols=partition_col,
            storage_options={
                "key": aws_auth_dict['AccessKeyId'],
                "secret": aws_auth_dict['SecretAccessKey'],
                "token": aws_auth_dict['SessionToken'],
            },
                )
else:
    df.to_parquet(
            AWS_S3_FOLDER+csv_file_name+'.parquet', 
            schema=pa_schema, 
            storage_options={
                "key": aws_auth_dict['AccessKeyId'],
                "secret": aws_auth_dict['SecretAccessKey'],
                "token": aws_auth_dict['SessionToken'],
            },
                )

In [10]:
#adicionando partições à metastore
from dataeng_services.session import Session
from datetime import datetime,timedelta

# In order to avoid exposing credentials in your code
# You may use dataeng-cli --refresh-token first to create/refresh a token
# Or provide env variables OLXBR_INTERNAL_AUTH_USERNAME and OLXBR_INTERNAL_AUTH_PASSWORD

from dataeng_services.session import Session

metaservice = Session().client("metaservice")

In [11]:
# Dados do processo no ETLaaS
from dateutil.relativedelta import *
schema_name = 'cross_strategy'
table_name = 'similarweb_demographics'

# Periodo
start_date = '2021-01-01'
end_date = '2023-12-01'
date_step = relativedelta(months=+1) # mensal
# date_step = timedelta(days = 1)  # diario

In [12]:
date_list = []
day = start_date
while datetime.strptime( day, '%Y-%m-%d') <= datetime.strptime( end_date, '%Y-%m-%d'):
    date_list.append( day )
    day = str( datetime.strptime( day, '%Y-%m-%d') + date_step )[:10]
#     day = day[:8]+'01' # mensal
# dt=2023-02-01/
    
date_list

['2021-01-01',
 '2021-02-01',
 '2021-03-01',
 '2021-04-01',
 '2021-05-01',
 '2021-06-01',
 '2021-07-01',
 '2021-08-01',
 '2021-09-01',
 '2021-10-01',
 '2021-11-01',
 '2021-12-01',
 '2022-01-01',
 '2022-02-01',
 '2022-03-01',
 '2022-04-01',
 '2022-05-01',
 '2022-06-01',
 '2022-07-01',
 '2022-08-01',
 '2022-09-01',
 '2022-10-01',
 '2022-11-01',
 '2022-12-01',
 '2023-01-01',
 '2023-02-01',
 '2023-03-01',
 '2023-04-01',
 '2023-05-01',
 '2023-06-01',
 '2023-07-01',
 '2023-08-01',
 '2023-09-01',
 '2023-10-01',
 '2023-11-01',
 '2023-12-01']

In [13]:
# Adding partitions
response = metaservice.add_partition(
    schema_name=schema_name,
    table_name=table_name,
    partition_names=partition_col,
    partition_values=[ [i] for i in date_list],
)

# Response
response.content

b'{"detail":"The partitions have been added to the requested table!"}'